# Un premier exemple!

Dans ce bloc-notes, nous introduirons les concepts de base de la simulation à événements discrets et son implémentation dans SimJulia.

Ce bloc-note est adapté de http://simjuliajl.readthedocs.io/en/stable/10_min/2_basic_concepts.html

En Julia, l'installation de SimJulia se fait à l'aide de la commande

In [ ]:
Pkg.add("SimJulia")

Pour mettre à jour les librairies, nous pourrons utiliser la commande

In [ ]:
Pkg.update()

In [ ]:
Par la suite, nous indiquerons que nous souhaitons utiliser SimJulia en indiquant

In [ ]:
using SimJulia

SimJulia travaille principalement avec l'approche par processus. Le comportement des éléments du système sont modélisés à l'aide de processus, qui se déroulent à l'intérieur d'un environnement. L'interaction des composantes entre elles et avec l'environnement se fait à partir d'événements.

Quand un processus pose un événement, ce processus est suspendu, jusqu'à ce que l'événement se produire. Plusieurs processus peuvent attendre le même événement. SimJulia les redémarre dans l'ordre dans lequel ils ont été mis en attente.

L'exemple suivant définit le comportement d'une voiture qui passe alternativement de l'état "stationnée" à l'état en marche. On voit plusieurs fonctions apparaître: now qui donne la valeur de l'horloge de simulation, yield qui met le processus en attente et Timeout qui permet de définir une durée dans le temps de la simulation.

In [ ]:
function car(env::Environment)
         while true
           println("Start parking at $(now(env))")
           parking_duration = 5.0
           yield(Timeout(env, parking_duration))
           println("Start driving at $(now(env))")
           trip_duration = 2.0
           yield(Timeout(env, trip_duration))
         end
       end

In [ ]:
env = Environment()

Crée le processus car qui doit être démarré et ajouté à l'environnement. Syntaxe: Process(env, func)

Dans SimJulia, un Process est techniquement un événement, héritant du type AbstractEvent.

In [ ]:
Process(env, car)

Lance la simulation. Le deuxième argument donne la durée de la simulation.

In [ ]:
run(env, 15.0)

Supposons à présent que nous avons une voiture électrique et qu'il faut prévoir de rechercher la voiture, ce que nous faisons avec la fonction

In [ ]:
function charge(env::Environment, duration::Float64)
         yield(Timeout(env, duration))
       end

Mettons à jour notre définition de processus et construisons la simulation de la même manière que précédemment.

In [ ]:
function car(env::Environment)
         while true
           println("Start parking and charging at $(now(env))")
           charge_duration = 5.0
           charge_proc = Process(env, charge, charge_duration)
           yield(charge_proc)
           println("Start driving at $(now(env))")
           trip_duration = 2.0
           yield(Timeout(env, trip_duration))
         end
       end

In [ ]:
env = Environment()

In [ ]:
Process(env, car)

In [ ]:
run(env, 15.0)